In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 读取数据集

调用框架中现有的API来读取数据
并且在实例化数据迭代器对象时，指定 batch_size 布尔值 is_train 表示是否希望数据迭代器每周期打乱

In [3]:
def load_array(data_arrays, batch_size, is_train = True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))
# next函数得到x 和 y

[tensor([[ 1.1691,  0.7391],
         [ 0.7515, -1.1461],
         [ 0.9376, -0.2137],
         [ 1.3757,  0.7201],
         [ 0.0142,  0.6263],
         [-1.8215, -0.7117],
         [ 0.9392, -2.9799],
         [ 0.0968,  0.8928],
         [ 0.3889, -0.8365],
         [-0.7993, -0.5303]]),
 tensor([[ 4.0306],
         [ 9.5920],
         [ 6.7938],
         [ 4.4964],
         [ 2.0954],
         [ 2.9754],
         [16.2058],
         [ 1.3580],
         [ 7.8187],
         [ 4.4090]])]

# 定义模型

1. 定义一个模型变量net （Sequential 类
2. 给定输入，从第一层->第二层->...

In [4]:
# "nn" 是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

### 初始化模型参数
深度学习框架通常有预定义的方法来初始化参数

这里 net 是sequential类， net[0]访问第一层

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数
计算均方误差使用的是MSELoss类，也称为平方 L2 范数。

In [6]:
loss = nn.MSELoss()

###  定义优化算法
PyTorch在optim模块中实现了该算法的许多变种。

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [8]:
# train
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

# loss predict
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

epoch 1, loss 0.000234
epoch 2, loss 0.000098
epoch 3, loss 0.000099
w的估计误差： tensor([-0.0004, -0.0010])
b的估计误差： tensor([0.0002])
